In [29]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'/opt/spark-2.4.5'

In [30]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf
from datetime import datetime
from sklearn.cluster import DBSCAN
from collections import Counter
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize']= 20,20
%matplotlib inline
import numpy as np
import pandas as pd
import os
import errno
 

In [31]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Word Count") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [32]:
path="date=2019-11-18.csv"

driver_sel_df=pd.read_csv(path)

In [33]:
driver_sel_df

,driver_id,tripid,points,Actualduration,AverageP,distance,duration,WaitingTime
0,0038263cc90dfa55d621a6425e46020136a318d12b9743...,191417796,50,526.0,10.520000,1.13,490.0,410.0
1,0038263cc90dfa55d621a6425e46020136a318d12b9743...,191428484,102,1020.0,10.000000,6.22,1010.0,758.0
2,0038263cc90dfa55d621a6425e46020136a318d12b9743...,191432146,138,1384.0,10.028986,5.28,1370.0,375.0
3,0038263cc90dfa55d621a6425e46020136a318d12b9743...,191439899,215,2190.0,10.186047,7.84,2179.0,265.0
4,0038263cc90dfa55d621a6425e46020136a318d12b9743...,191446799,69,686.0,9.942029,3.00,680.0,292.0
...,...,...,...,...,...,...,...,...
1098,910d26c1e71d01526f727c8d9d24f5318f152bb288d307...,191410555,143,1440.0,10.069930,10.90,1420.0,92.0
1099,910d26c1e71d01526f727c8d9d24f5318f152bb288d307...,191418454,90,899.0,9.988889,3.94,890.0,609.0
1100,910d26c1e71d01526f727c8d9d24f5318f152bb288d307...,191433213,56,563.0,10.053571,0.52,550.0,433.0
1101,910d26c1e71d01526f727c8d9d24f5318f152bb288d307...,191437309,151,1507.0,9.980132,7.20,1500.0,493.0


In [34]:
driver_sel_df = driver_sel_df[driver_sel_df['AverageP'] <= 11]
driver_sel_df = driver_sel_df[driver_sel_df["distance"] > 0.05]
driver_sel_df = driver_sel_df[driver_sel_df["duration"] > 0]
driver_sel_df = driver_sel_df[driver_sel_df["WaitingTime"] > 0]
driver_sel_df = driver_sel_df[driver_sel_df["points"] > 0]

In [35]:
driver_sel_df

,driver_id,tripid,points,Actualduration,AverageP,distance,duration,WaitingTime
0,0038263cc90dfa55d621a6425e46020136a318d12b9743...,191417796,50,526.0,10.520000,1.13,490.0,410.0
1,0038263cc90dfa55d621a6425e46020136a318d12b9743...,191428484,102,1020.0,10.000000,6.22,1010.0,758.0
2,0038263cc90dfa55d621a6425e46020136a318d12b9743...,191432146,138,1384.0,10.028986,5.28,1370.0,375.0
3,0038263cc90dfa55d621a6425e46020136a318d12b9743...,191439899,215,2190.0,10.186047,7.84,2179.0,265.0
4,0038263cc90dfa55d621a6425e46020136a318d12b9743...,191446799,69,686.0,9.942029,3.00,680.0,292.0
...,...,...,...,...,...,...,...,...
1098,910d26c1e71d01526f727c8d9d24f5318f152bb288d307...,191410555,143,1440.0,10.069930,10.90,1420.0,92.0
1099,910d26c1e71d01526f727c8d9d24f5318f152bb288d307...,191418454,90,899.0,9.988889,3.94,890.0,609.0
1100,910d26c1e71d01526f727c8d9d24f5318f152bb288d307...,191433213,56,563.0,10.053571,0.52,550.0,433.0
1101,910d26c1e71d01526f727c8d9d24f5318f152bb288d307...,191437309,151,1507.0,9.980132,7.20,1500.0,493.0


In [36]:
s=spark.sparkContext

In [37]:
sql_c=SQLContext(s)

In [38]:
driver_gps_df =spark.read.orc('/FYP/huntersfyp17/date=2019-11-18/part*')


In [39]:
driver_gps_df.show(1)

+--------------------+------+----------+-------------+-------------+-------------+
|           driver_id|status| timestamp|     latitude|    longitude|vehicle_model|
+--------------------+------+----------+-------------+-------------+-------------+
|fb26acdfa6d27c156...|     F|1574037106|6.71144580841|79.9109191895|            4|
+--------------------+------+----------+-------------+-------------+-------------+
only showing top 1 row



In [40]:
trip = sql_c.read.orc('/FYP/huntersfyp17/dropdatesk=20191118/part*')

In [41]:
def getDriverTripDF(driverid):
    driver_td_df=trip.select(["tripid","driverid","pickuplatitude","pickuplongitude","droplatitude","droplongitude", "actualpickuptime","droptime"]).where(trip.driverid == driverid ).sort("tripid").collect()
    columns=["tripid","driverid","pickuplatitude","pickuplongitude","droplatitude","droplongitude", "actualpickuptime","droptime"]
    df_trip_driver = pd.DataFrame(driver_td_df, columns=columns)
    
    return df_trip_driver

In [42]:
def getDriverGPSpoints(driverid):
    driver_gps=driver_gps_df.select(["driver_id","timestamp","latitude","longitude"]).where(driver_gps_df.driver_id == driverid ).sort("timestamp").collect()    
    columns2=["driver_id","timestamp","latitude","longitude"]
    df_gps = pd.DataFrame(driver_gps, columns=columns2)
    for num in df_gps.index:
        df_gps.loc[num,'time'] = datetime.fromtimestamp(df_gps.loc[num,'timestamp'])
    return df_gps


In [43]:
#assigning relevant tripid to traces
def assign_trip_id(df_gps_driver,df_trip_driver):
    df_gps_driver['tripid']= ' '
    for index in df_gps_driver.index :
        for num in df_trip_driver.index:
            if df_trip_driver['actualpickuptime'][num] < df_gps_driver["time"][index] and df_gps_driver["time"][index]<df_trip_driver['droptime'][num]:
                df_gps_driver.loc[index, 'tripid'] =  df_trip_driver['tripid'][num]
    df_gps_driver.drop(df_gps_driver[df_gps_driver['tripid'] ==' '].index, inplace = True)
    return df_gps_driver


In [44]:
path="driver.txt"
f = open(path, "r")
driverlist=f.readlines()

In [45]:
driverlist

['...\n',
 ...]

In [46]:
selected_trip_list=driver_sel_df.tripid.unique()

In [47]:
def getTripGPSFile(driver_id):
    df_trip_driver=getDriverTripDF(driver_id)
    df_gps_driver=getDriverGPSpoints(driver_id)
    df_gps_driver_final=assign_trip_id(df_gps_driver,df_trip_driver)
    co=df_trip_driver[df_trip_driver.columns[0]].count() 
    if co!=0:
        day_trips=df_gps_driver_final.tripid.unique()
        for atrip in selected_trip_list:
            if atrip in day_trips:
                tripfile=df_gps_driver_final[df_gps_driver_final["tripid"]==atrip]
                file_name=str("Trip_GPS_CSV/"+str(driver_id)+"/"+str(atrip)+".csv")
                isDirectory = os.path.isdir("Trip_GPS_CSV/"+str(driver_id))
                if not isDirectory:
                    os.mkdir("Trip_GPS_CSV/"+str(driver_id))
                tripfile.to_csv(file_name)
                print(file_name)
                
    

In [ ]:
count=0
for j in driverlist[0:500]:
    count+=1 
    getTripGPSFile(j[:-1])
    print(count)

Trip_GPS_CSV/0038263cc90dfa55d621a6425e46020136a318d12b974340f518892a6ef690ff/191417796.csv
Trip_GPS_CSV/0038263cc90dfa55d621a6425e46020136a318d12b974340f518892a6ef690ff/191428484.csv
Trip_GPS_CSV/0038263cc90dfa55d621a6425e46020136a318d12b974340f518892a6ef690ff/191432146.csv
Trip_GPS_CSV/0038263cc90dfa55d621a6425e46020136a318d12b974340f518892a6ef690ff/191439899.csv
Trip_GPS_CSV/0038263cc90dfa55d621a6425e46020136a318d12b974340f518892a6ef690ff/191446799.csv
1
2
3
Trip_GPS_CSV/00ce195f11cfa5d5e228095893a866e2ec79cf3aa1c423d869485a5d06226872/191430779.csv
Trip_GPS_CSV/00ce195f11cfa5d5e228095893a866e2ec79cf3aa1c423d869485a5d06226872/191441994.csv
Trip_GPS_CSV/00ce195f11cfa5d5e228095893a866e2ec79cf3aa1c423d869485a5d06226872/191443620.csv
4
5
Trip_GPS_CSV/01af51ec6c430f406fcde188aa86412b68870084ccda59ac34ddcbef3ce62628/191421199.csv
Trip_GPS_CSV/01af51ec6c430f406fcde188aa86412b68870084ccda59ac34ddcbef3ce62628/191443611.csv
Trip_GPS_CSV/01af51ec6c430f406fcde188aa86412b68870084ccda59ac34ddcbef3

Trip_GPS_CSV/0f3c75cc799f636d7c4b858b856ecc5abd2a04fec49024e85e5f23741a74e180/191403432.csv
Trip_GPS_CSV/0f3c75cc799f636d7c4b858b856ecc5abd2a04fec49024e85e5f23741a74e180/191420949.csv
Trip_GPS_CSV/0f3c75cc799f636d7c4b858b856ecc5abd2a04fec49024e85e5f23741a74e180/191428711.csv
Trip_GPS_CSV/0f3c75cc799f636d7c4b858b856ecc5abd2a04fec49024e85e5f23741a74e180/191438987.csv
Trip_GPS_CSV/0f3c75cc799f636d7c4b858b856ecc5abd2a04fec49024e85e5f23741a74e180/191467838.csv
48
49
Trip_GPS_CSV/1044aa67ebb6748c198cb87c2e0e467e2701c7296dc8d5a49892ba730ed6b879/191437749.csv
Trip_GPS_CSV/1044aa67ebb6748c198cb87c2e0e467e2701c7296dc8d5a49892ba730ed6b879/191439638.csv
Trip_GPS_CSV/1044aa67ebb6748c198cb87c2e0e467e2701c7296dc8d5a49892ba730ed6b879/191446888.csv
Trip_GPS_CSV/1044aa67ebb6748c198cb87c2e0e467e2701c7296dc8d5a49892ba730ed6b879/191453390.csv
50
51
Trip_GPS_CSV/10768682d2911c0a79d67e93d9639d18eb6223c101ec2a65786318b0941cdc01/191431962.csv
Trip_GPS_CSV/10768682d2911c0a79d67e93d9639d18eb6223c101ec2a65786318b

In [ ]:
Spark.stop()